# Mario data on Neo4j

In [5]:
import pandas as pd
from neo4j import GraphDatabase
import os
from dotenv import load_dotenv

In [6]:
# Load environment variables from .env file
load_dotenv()
# Ensure the .env file is in the same directory as this script
if not os.path.exists('.env'):
    raise FileNotFoundError("The .env file is missing. Please create it with the necessary credentials.")

# --- Configuration ---
AURA_URI = os.getenv("NEO4J_URI")
AURA_USER = os.getenv("NEO4J_USERNAME")
AURA_PASSWORD = os.getenv("NEO4J_PASSWORD")

In [28]:
class MarioGraphDBImporter:
    """
    A class to import Mario franchise data from CSV files into a Neo4j database.
    """

    def __init__(self, uri, user, password):
        """
        Initializes the importer and connects to the Neo4j database.
        """
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
        self.driver.verify_connectivity()
        print("✅ Successfully connected to Neo4j AuraDB.")

    def cleanup_database(self):
        """
        Deletes all nodes and relationships from the database for a clean import.
        """
        print("🧹 Deleting all existing data from the database...")
        with self.driver.session(database="neo4j") as session:
            session.run("MATCH (n) DETACH DELETE n")
        print("✅ Database is now empty.")
        
    def close(self):
        """
        Closes the database connection.
        """
        self.driver.close()
        print("✅ Database connection closed.")

    def setup_constraints(self):
        """
        Sets up unique constraints on node labels for data integrity and performance.
        """
        with self.driver.session(database="neo4j") as session:
            # Create unique constraints for Game (more specific for game releases) and Character nodes
            session.run("CREATE CONSTRAINT IF NOT EXISTS FOR (g:Game) REQUIRE g.gameId IS UNIQUE")
            session.run("CREATE CONSTRAINT IF NOT EXISTS FOR (c:Character) REQUIRE c.name IS UNIQUE")
            print("✅ Constraints created for :Game(gameId) and :Character(name).")

            # Add additional indexes for common query patterns
            session.run("CREATE INDEX IF NOT EXISTS FOR (g:Game) ON (g.year)")
            session.run("CREATE INDEX IF NOT EXISTS FOR (g:Game) ON (g.console)")
            session.run("CREATE INDEX IF NOT EXISTS FOR (c:Character) ON (c.species)")
            print("✅ Additional indexes created for :Game(year), :Game(console), and :Character(species).")


    def import_nodes_and_game_relations(self, merged_data_path):
        """
        Imports Game and Character nodes and creates distinct relationship types
        (BOSS_IN, ENEMY_IN, CHARACTER_IN) between them.
        Each game release (name, console, year) is treated as a unique Game node.
        """
        df = pd.read_csv(merged_data_path)
        df.dropna(subset=['Game', 'Figure', 'Relation', 'Console', 'Year'], inplace=True)

        # Replace NaN in 'Sales' with 0 before importing, a common approach which cleans data at the source
        # In integration part, the NaN value correctly signifies that
        # sales data was "not found" or "not applicable" during the merge to reflect the state of the data.
        # Now, in storage step, we replace NaN with 0 to ensure that the database
        # can handle these values without issues
        df['Sales'] = df['Sales'].fillna(0)

        # Convert the 'Species' column into a list of strings
        df['Species'] = df['Species'].fillna('').apply(lambda x: [s.strip() for s in str(x).split(';') if s.strip()])

        print(f" Importing {len(df)} relationships from '{merged_data_path}'...")
        records = df.to_dict('records')

        # This query uses APOC to create relationships with dynamic types
        # It first creates a unique game identifier to treat each release as a distinct node
        query = """
        UNWIND $records AS row
        // Create a unique ID for each game release
        WITH row, row.Game + '_' + row.Console + '_' + row.Year AS gameId

        // Create or find the Game node based on the unique gameId
        MERGE (g:Game {gameId: gameId})
        ON CREATE SET
            g.name = row.Game,
            g.year = toInteger(row.Year),
            g.console = row.Console,
            g.sales = toFloat(row.Sales)
        ON MATCH SET
            // If the game already exists, update all its properties
            // to reflect any changes in the source CSV.
            g.name = row.Game,
            g.year = toInteger(row.Year),
            g.console = row.Console,
            g.sales = toFloat(row.Sales)

        // Create or find the Character node
        MERGE (c:Character {name: row.Figure})
        ON CREATE SET
            c.species = row.Species
        ON MATCH SET
            // Only update species if the new list from the CSV is not empty
            // This prevents overwriting existing data with an empty list
            c.species = CASE WHEN size(row.Species) > 0 THEN row.Species ELSE c.species END

        // Explicitly pass variables to the next part of the query
        WITH c, g, row

        // Use apoc.merge.relationship to create a relationship
        // where the type is taken directly from the 'Relation' column.
        CALL apoc.merge.relationship(c, row.Relation, {}, {}, g) YIELD rel
        RETURN count(rel)
        """
        # apoc is used to safely create the relationship only if it doesn't already exist
        # Then, AuraDB includes the APOC library by default

        with self.driver.session(database="neo4j") as session:
            session.run(query, records=records)
        print("✅ Finished importing games, characters, and their distinct roles.")


    def import_character_relations(self, api_data_path):
        """
        Imports FRIEND_WITH and ENEMY_WITH relationships between characters.
        """
        df = pd.read_csv(api_data_path)
        df.dropna(subset=['Figure', 'RelatedCharacter', 'RelationshipType'], inplace=True)
        
        # Convert species columns into a list of strings, handling NaNs
        df['FigureSpecies'] = df['FigureSpecies'].fillna('').apply(lambda x: [s.strip() for s in str(x).split(';') if s.strip()])
        df['RelatedCharacterSpecies'] = df['RelatedCharacterSpecies'].fillna('').apply(lambda x: [s.strip() for s in str(x).split(';') if s.strip()])
        
        print(f" Importing {len(df)} character relationships from '{api_data_path}'...")
        
        records = df.to_dict('records')

        query = """
        UNWIND $records AS row
        // Merge the first character, setting species on creation
        MERGE (c1:Character {name: row.Figure})
        ON CREATE SET c1.species = row.FigureSpecies
        ON MATCH SET
          // Only set species if it's currently null and the new data is not empty
          c1.species = CASE WHEN c1.species IS NULL AND size(row.FigureSpecies) > 0 THEN row.FigureSpecies ELSE c1.species END

        // Merge the second character, setting species on creation
        MERGE (c2:Character {name: row.RelatedCharacter})
        ON CREATE SET c2.species = row.RelatedCharacterSpecies
        ON MATCH SET
          // Only set species if it's currently null and the new data is not empty
          c2.species = CASE WHEN c2.species IS NULL AND size(row.RelatedCharacterSpecies) > 0 THEN row.RelatedCharacterSpecies ELSE c2.species END

        // Explicitly pass variables to the next part of the query
        WITH c1, c2, row
        
        CALL apoc.merge.relationship(c1, row.RelationshipType, {}, {}, c2) YIELD rel
        RETURN count(rel)
        """
        
        with self.driver.session(database="neo4j") as session:
            session.run(query, records=records)
        print("✅ Finished importing character-to-character relationships.")

In [29]:
# Create an importer instance with your credentials
importer = MarioGraphDBImporter(AURA_URI, AURA_USER, AURA_PASSWORD)

# 0. Clean the database before importing (for a fresh start)
importer.cleanup_database()

# 1. Set up database constraints (run this first)
importer.setup_constraints()

# 2. Import data from the first CSV file
importer.import_nodes_and_game_relations('data/merged_data.csv')

# 3. Import data from the second CSV file
importer.import_character_relations('data/merged_data_API.csv')

# 4. Close the connection
importer.close()

✅ Successfully connected to Neo4j AuraDB.
🧹 Deleting all existing data from the database...
✅ Database is now empty.
✅ Constraints created for :Game(gameId) and :Character(name).
✅ Additional indexes created for :Game(year), :Game(console), and :Character(species).
 Importing 6959 relationships from 'data/merged_data.csv'...
✅ Finished importing games, characters, and their distinct roles.
 Importing 2524 character relationships from 'data/merged_data_API.csv'...
✅ Finished importing character-to-character relationships.
✅ Database connection closed.


# Queries

In [30]:
import plotly.express as px

# --- Database Connection ---
# This function will help run queries and return the results as a DataFrame
def run_query(driver, query):
    """
    Executes a Cypher query and returns the result as a pandas DataFrame.
    """
    with driver.session(database="neo4j") as session:
        result = session.run(query)
        return pd.DataFrame([r.data() for r in result])

# Establish the connection driver
try:
    driver = GraphDatabase.driver(AURA_URI, auth=(AURA_USER, AURA_PASSWORD))
    driver.verify_connectivity()
    print("✅ Successfully connected to Neo4j AuraDB.")
except Exception as e:
    print(f"❌ Failed to connect to Neo4j: {e}")


✅ Successfully connected to Neo4j AuraDB.


## Query 1

### What are the top 10 best-selling games?

In [31]:
# Define the Cypher query
top_sales_query = """
MATCH (g:Game)
// Explicitly group by game name and aggregate the sales.
// Since all releases of a game title share the same sales figure, max() is a safe choice.
RETURN g.name AS Game, max(g.sales) AS SalesInMillions
ORDER BY SalesInMillions DESC
LIMIT 10
"""

# Run the query and get the DataFrame
df_sales = run_query(driver, top_sales_query)

# Display the tabular data
print("--- Top 10 Best-Selling Games ---")
display(df_sales)

# --- Visualize the results ---
# Sorting the dataframe makes the horizontal bar chart display correctly (highest at top)
df_sales_sorted = df_sales.sort_values('SalesInMillions', ascending=True)

fig = px.bar(
    df_sales_sorted,
    x="SalesInMillions",
    y="Game",
    range_x=[0, df_sales_sorted['SalesInMillions'].max() * 1.1],  # Add some padding to the x-axis
    orientation='h',
    title="Top 10 Best-Selling Mario Games",
    labels={'SalesInMillions': 'Sales (in millions)', 'Game': 'Game Title'},
    text="SalesInMillions",
    template="plotly_white"
)

# Update layout for a cleaner look
fig.update_traces(
    texttemplate='%{text:.2f}M',
    textposition='outside',
    # Consistent color scale for better visualization
    marker_color="#5664ff"  # Using a single color for all bars
)
fig.update_layout(
    showlegend=False,
    yaxis={'categoryorder':'total ascending'},
)
fig.show()

--- Top 10 Best-Selling Games ---


,Game,SalesInMillions
0,super mario bros.,45.31
1,tetris,35.84
2,mario kart wii,35.52
3,new super mario bros.,29.80
4,new super mario bros. wii,28.32
5,super mario world,26.07
6,mario kart ds,23.21
7,super mario bros. 3,22.48
8,super mario 64,22.19
9,super mario land,18.14


## Query 2

### Who are the characters with the most game appearances?

In [32]:
# Define the Cypher query
# Made distinct to avoid duplicates in case of multiple appearances
chars_appearances_query = """
MATCH (c:Character)-[]->(g:Game)
RETURN c.name AS Character, count(DISTINCT g) AS Appearances
ORDER BY Appearances DESC
LIMIT 10
"""

# Run the query and get the DataFrame
df_appearances = run_query(driver, chars_appearances_query)

# Display the tabular data
print("\n--- Top 10 Characters with most appearances ---")
display(df_appearances)

# --- Visualize the results ---
# Sorting the dataframe ascending makes the horizontal bar chart display correctly (highest at top)
df_appearances_sorted = df_appearances.sort_values('Appearances', ascending=True)

fig = px.bar(
    df_appearances_sorted,
    x="Appearances",
    y="Character",
    orientation='h',
    title="Top 10 Characters by Number of Game Appearances",
    labels={'Appearances': 'Number of Game Appearances', 'Character': 'Character'},
    template="plotly_white"
)

# Customize the layout
fig.update_traces(
    textposition='outside',
    # Consistent color scale for better visualization
    marker_color="#5664ff"  # Using a single color for all bars
)

# Combine layout updates and add padding to the x-axis to prevent text labels from being cut off
fig.update_layout(
    showlegend=False,
    yaxis={'categoryorder':'total ascending'},
    xaxis_range=[0, df_appearances_sorted['Appearances'].max() * 1.1]
)
fig.show()



--- Top 10 Characters with most appearances ---


,Character,Appearances
0,mario,278
1,luigi,223
2,princess peach,186
3,yoshi,180
4,bowser,167
5,donkey kong,145
6,toad,97
7,waluigi,81
8,birdo,80
9,boo,77


Stop querying by closing the DB

In [33]:
# Close the driver connection
driver.close()
print("\n✅ Database connection closed.")


✅ Database connection closed.
